In [22]:
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [27]:
datadir = joinpath("testcase") 
gens = CSV.read(joinpath(datadir,"Gen36.csv"), DataFrame);

In [28]:
gens

Row,Zone,connnode,c2,c1,c0,PgMax,PgMin,RgMax,RgMin,PgPrev
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,1,1,0,105,0,130.0,0,100,-100,0
2,1,2,0,120,0,1516.8,0,100,-100,0
3,1,3,0,70,0,51.0,0,100,-100,0
4,1,4,0,12,0,101.8,0,100,-100,0
5,1,5,0,5,0,510.3,0,100,-100,0
6,2,6,0,130,0,152.0,0,100,-100,0
7,2,7,0,70,0,54.0,0,100,-100,0
8,2,8,0,65,0,1000.0,0,100,-100,0
9,2,9,0,12,0,107.7,0,100,-100,0


In [29]:
gdf = groupby(gens, :Zone)

Row,Zone,connnode,c2,c1,c0,PgMax,PgMin,RgMax,RgMin,PgPrev
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,1,1,0,105,0,130.0,0,100,-100,0
2,1,2,0,120,0,1516.8,0,100,-100,0
3,1,3,0,70,0,51.0,0,100,-100,0
4,1,4,0,12,0,101.8,0,100,-100,0
5,1,5,0,5,0,510.3,0,100,-100,0
Row,Zone,connnode,c2,c1,c0,PgMax,PgMin,RgMax,RgMin,PgPrev
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,36,148,0,70,0,10.0,0,100,-100,0
2,36,149,0,12,0,20.4,0,100,-100,0


In [30]:
gdf

Row,Zone,connnode,c2,c1,c0,PgMax,PgMin,RgMax,RgMin,PgPrev
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,1,1,0,105,0,130.0,0,100,-100,0
2,1,2,0,120,0,1516.8,0,100,-100,0
3,1,3,0,70,0,51.0,0,100,-100,0
4,1,4,0,12,0,101.8,0,100,-100,0
5,1,5,0,5,0,510.3,0,100,-100,0
Row,Zone,connnode,c2,c1,c0,PgMax,PgMin,RgMax,RgMin,PgPrev
,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64
1,36,148,0,70,0,10.0,0,100,-100,0
2,36,149,0,12,0,20.4,0,100,-100,0


In [31]:
keys(gdf)

36-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (Zone = 1,)
 GroupKey: (Zone = 2,)
 GroupKey: (Zone = 3,)
 GroupKey: (Zone = 4,)
 GroupKey: (Zone = 5,)
 GroupKey: (Zone = 6,)
 GroupKey: (Zone = 7,)
 GroupKey: (Zone = 8,)
 GroupKey: (Zone = 9,)
 GroupKey: (Zone = 10,)
 GroupKey: (Zone = 11,)
 GroupKey: (Zone = 12,)
 GroupKey: (Zone = 13,)
 ⋮
 GroupKey: (Zone = 25,)
 GroupKey: (Zone = 26,)
 GroupKey: (Zone = 27,)
 GroupKey: (Zone = 28,)
 GroupKey: (Zone = 29,)
 GroupKey: (Zone = 30,)
 GroupKey: (Zone = 31,)
 GroupKey: (Zone = 32,)
 GroupKey: (Zone = 33,)
 GroupKey: (Zone = 34,)
 GroupKey: (Zone = 35,)
 GroupKey: (Zone = 36,)